# Ансамбли деревьев решений
*Ensembles* - это методы, которые сочетают в себе множество моделей машинного обучения, чтобы в итоге дать ещё более мощную модель.

Выделяют две основные ансамблевые модели
## Случайный лес
Как упоминалось ранее, деревья склонны к переобучению. Случайный лес призван решить эту проблему. Он представляет собой набор деревьев решений, где каждое дерево немного отличается от других. Причем каждое дерево может довольного хорошо прогнозировать, но **переобучаются на некоторой части данных**. Используя много таких деревьев, мы уменьшаем риск переобучиться путем усреднения результатов обучения всех деревьев. 

In [56]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10)

Параметр `n_estimators` характеризует количество деревьев, входящих в лес. Эти деревья будут построены совершенно независимо друг от друга, и алгоритм будет случайным образом отбирать признаки для построения каждого дерева.

**bootstrap sample** - бутстреп-выборка - выборка того же размера, что и исходная, но в ней некоторые примеры будут отсутствовать, а некоторые попадут несколько раз. На основе сформированной bootstrap sample строится дерево решений следующим образом: для очередного разбиения узла случайным образом выбирается подмножество признаков, затем находится наилучший тест для этого подмножества. Мощность отбираемого подмножество регулируется параметром `max_features`.

bootstrap sample + отбираемое подмножество признаков вместе приводят к тому, что все деревья в случайном лесе отличаются друг от друга, т.к. они обучаются на немного разных наборах данных и используют для обучения различные (относительно) признаки.

*Замечание*: высокое значение `max_features` означает, что деревья в лесу легко аппроксимируют данные и мало чем отличаются; низкое значение `max_features` означает, что деревья сильно отличаются и, возможно, каждое имеет большую глубину, чтобы хорошо соответствовать данным.

In [57]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    random_state=0)
model.fit(X_train, y_train)

print(f'Всего признаков в наборе: {cancer.data.shape[1]}')
print(f'Количество деревьев в лесу: {model.n_estimators}')
print(f'Мощность отбираемого подмножества: {model.max_features}')
print(f'Правильность на тестовом наборе: {model.score(X_test, y_test)}')

Всего признаков в наборе: 30
Количество деревьев в лесу: 10
Мощность отбираемого подмножества: auto
Правильность на тестовом наборе: 0.951048951048951


Чтобы дать прогноз для случайного леса, алгоритм сначала делает прогноз для каждого дерева. Для регрессии результаты целевой переменной могут быть усреднены, а для классификации: каждое дерево вычисляет вероятности для принадлежности к каждому из классов, а итоговый класс получается путем усреднения наблюдений по всем деревьям.

In [58]:
model = RandomForestClassifier(n_estimators=55, max_features=20, random_state=10)
model.fit(X_train, y_train)

print(f'Всего признаков в наборе: {cancer.data.shape[1]}')
print(f'Количество деревьев в лесу: {model.n_estimators}')
print(f'Мощность отбираемого подмножества: {model.max_features}')
print(f'Правильность на тестовом наборе: {model.score(X_test, y_test)}')

Всего признаков в наборе: 30
Количество деревьев в лесу: 55
Мощность отбираемого подмножества: 20
Правильность на тестовом наборе: 0.986013986013986


### Резюме
**Случайный лес** - одна из самых широко используемых моделей, поскольку она часто дает хорошее качество без утомительной настройки параметров и не требует нормализации данных.
По сути случайный лес обладает всеми преимуществами деревьев, хотя не лишен недостатков:  
- трудно интерпретировать результат для неспециалистов  
- установка показателя `random_state` может значительно влиять на построение модели, поскольку случайный лес в построении использует генератор псевдослучайных чисел  
- плохо работает на данных высокой размерности (например, на текстовых).
Случайный лес требует много памяти, медленно обучается, поэтому, если требуется сохранение памяти и скорости, лучше рассмотреть другую модель (например, линейную).

Обычно чем больше  число деревьев в лесу (`n_estimators`), тем лучше. Но обратная сторона увеличения деревьев в том, что с ростом количества деревьев требуется больше памяти и больше времени для обучения. Параметр `n_jobs` устанавливает количество ядер, которые используются при построении (они приводят к линейному росту: 2 ядра -> в 2 раза быстрее). Чтобы использовать все ядра компьютера, нужно установить `n_jobs=-1`.

## Градиентный бустинг деревьев регрессии
Ещё один ансамблевый метод, который объединяет в себе множество деревьев для создания более мощной модели. Эту модель (несмотря на название) можно использовать как для регрессии, так и для классификации.
В градиентном бустинге строится последовательность деревьев, в которой каждое дерево пытается "исправить" ошибки предыдущего. По умолчанию в градиентном бустинге отсутствует случайность, вместо этого используется предварительная обрезка. Обычно деревья градиентного бустинга небольшие по глубине, что делает модель легче с точки зрения памяти и скорости вычислений.
Идея градиентного бустинга - объединение простых (по отдельности слабых - weak learners) деревьев. Каждое дерево дает хорошие прогнозы только для части данных. Поэтому для улучшения нужно "добавить" несколько деревьев.

In [132]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=75,
                                   learning_rate=0.45,
                                   random_state=15,
                                  max_depth=1)
model.fit(X_train, y_train)

print(f'Правильности градиентного бустинга на тестовой выборке: {model.score(X_test, y_test)}')

Правильности градиентного бустинга на тестовой выборке: 0.986013986013986


 `learning_rate` контролирует насколько сильно каждое дерево будет пытаться исправить ошибки предыдущего: высокое значение говорит о том, что каждое дерево вносит большие корректировки (то есть модель становится более сложной)  
 `max_depth` контролирует глубину деревьев, входящих в бустинг
 
 ### Резюме
 Градиентный бустинг также является одним из самых мощных и широко используемых приемов. Его основной недостаток в том, что он требует тщательной настройки параметров, но зато он обучается без необходимости нормализации и показывает отличные результаты. Его важной составляющие является соотношение параметров `n_estimators` и `learning_rate`. Градиентный бустинг, как и другие модели на деревьях, не очень хорошо для сильно разреженных данных.